In [133]:
from xbbg import blp
from blp import blp as bp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime, timedelta, time
from zoneinfo import ZoneInfo
from matplotlib.patches import Patch


In [ ]:
# Geneva timezone (handles CEST/CET automatically)
geneva_tz = ZoneInfo("Europe/Zurich")

# Get current time in Geneva
now_geneva = datetime.now(geneva_tz)

# Set 2:00 AM today in Geneva
start_geneva = (now_geneva - timedelta(days=3)).replace(hour=2, minute=0, second=0, microsecond=0)

#start_geneva = now_geneva - timedelta(hours=72) # use this for showing the last 3 days

# Convert both to UTC
start_datetime = start_geneva.astimezone(ZoneInfo("UTC")).strftime('%Y-%m-%dT%H:%M:%S')
end_datetime = now_geneva.astimezone(ZoneInfo("UTC")).strftime('%Y-%m-%dT%H:%M:%S')

#Get intraday data for both instruments
def get_intraday_data(ticker, start_dt, end_dt):
    bquery = bp.BlpQuery().start()
    return bquery.bdib(
        ticker,
        event_type="ASK",
        interval=1,
        start_datetime=start_dt,
        end_datetime=end_dt,
    )
    

In [135]:
def classify_regime(df, lookback=10):
    df = df.copy()
    df['chg_2y'] = df['close_2y'].diff(lookback)
    df['chg_10y'] = df['close_10y'].diff(lookback)

    conditions = [
        (df['chg_2y'] < 0) & (df['chg_10y'] <= 0) & (df['chg_10y'] > df['chg_2y']),
        (df['chg_2y'] >= 0) & (df['chg_10y'] > 0) & (df['chg_10y'] > df['chg_2y']),
        (df['chg_2y'] <= 0) & (df['chg_10y'] < 0) & (df['chg_10y'] < df['chg_2y']),
        (df['chg_2y'] > 0) & (df['chg_10y'] >= 0) & (df['chg_10y'] < df['chg_2y']),
        (df['chg_2y'] > 0) & (df['chg_10y'] < 0),
        (df['chg_2y'] < 0) & (df['chg_10y'] > 0)
    ]

    labels = ['BullSteepener', 'BearSteepener', 'BullFlattener',
              'BearFlattener', 'TwistFlattener', 'TwistSteepener']

    df['regime'] = np.select(conditions, labels, default='Other')
    return df


In [ ]:
import plotly.graph_objects as go
import pandas as pd

def plot_yield_curve_regimes_plotly(df, curve_name: str = "") -> go.Figure:
    """
    Plot 2s10s curve with colored underfills for each regime.
    """
    df = df.copy()
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Compute spread
    df['2s10s'] = df['close_10y'] - df['close_2y']
    df = df.dropna(subset=['2s10s'])

    # Regime colors
    regime_colors = {
        'BullSteepener': 'green',
        'BearSteepener': 'red',
        'BullFlattener': 'deepskyblue',
        'BearFlattener': 'orange',
        'TwistFlattener': 'violet',
        'TwistSteepener': 'gold',
        'Unchanged': 'grey'
    }

    ymin = float(df['2s10s'].min()) - 0.01
    ymax = float(df['2s10s'].max()) + 0.01

    fig = go.Figure()

    # Add filled regions by regime
    for i in range(1, len(df)):
        t_prev = df.index[i - 1]
        t_curr = df.index[i]
        y_prev = df['2s10s'].iloc[i - 1]
        y_curr = df['2s10s'].iloc[i]
        regime = df['regime'].iloc[i]
        color = regime_colors.get(regime, 'grey')

        fig.add_trace(
            go.Scatter(
                x=[t_prev, t_curr],
                y=[y_prev, y_curr],
                mode="lines",
                line=dict(color=color, width=0.5),
                fill="tozeroy",       # fill under the line
                fillcolor=color,
                opacity=0.4,
                name=regime,
                showlegend=False      # avoid duplicate legend spam
            )
        )

    # Overlay the white 2s10s line on top
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['2s10s'],
            mode='lines',
            name='2s10s',
            line=dict(width=2, color='white')
        )
    )

    # Dummy legend entries for regimes
    for label, color in regime_colors.items():
        fig.add_trace(
            go.Scatter(
                x=[None], y=[None],
                mode="lines",
                line=dict(color=color, width=10),
                name=label,
                showlegend=True
            )
        )

    # Style
    fig.update_layout(
        title=dict(
            text=f"{curve_name} 2s10s Yield Curve (10-min lookback)",
            font=dict(color="white", size=16),
            x=0.5, xanchor="center"
        ),
        paper_bgcolor="black",
        plot_bgcolor="black",
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1.0,
            xanchor="left",
            x=0.01,
            font=dict(color="white", size=10),
            bgcolor="rgba(0,0,0,0)"
        ),
        margin=dict(l=60, r=20, t=60, b=40),
        hovermode="x unified"
    )

    fig.update_xaxes(showgrid=False, zeroline=False, color="white")
    fig.update_yaxes(range=[ymin, ymax], showgrid=False, zeroline=False, color="white")

    return fig


In [137]:
#Pull intraday data
print("Fetching 2Y ESTR data...")
ESTR_2y = get_intraday_data("EESWE2 Curncy", start_datetime, end_datetime)
print("Fetching 10Y ESTR data...")
ESTR_10y = get_intraday_data("EESWE10 Curncy", start_datetime, end_datetime)

#Merge dataframes on time
ESTR_combined = pd.merge(ESTR_2y[['time', 'close']], ESTR_10y[['time', 'close']], 
                      on='time', suffixes=('_2y', '_10y'))

#Set time as index
ESTR_combined['time'] = pd.to_datetime(ESTR_combined['time'], utc=True)
ESTR_combined.set_index('time', inplace=True)
ESTR_combined.index = ESTR_combined.index.tz_convert('Europe/Zurich')


Fetching 2Y ESTR data...
Fetching 10Y ESTR data...


In [138]:
plot_yield_curve_regimes_plotly(classify_regime(ESTR_combined), curve_name = "ESTR")

In [139]:
#Pull intraday data
print("Fetching 2Y UST data...")
UST_2y = get_intraday_data("USGG2YR Index", start_datetime, end_datetime)
print("Fetching 10Y UST data...")
UST_10y = get_intraday_data("USGG10YR Index", start_datetime, end_datetime)

#Merge dataframes on time
UST_combined = pd.merge(UST_2y[['time', 'close']], UST_10y[['time', 'close']], 
                      on='time', suffixes=('_2y', '_10y'))

#Set time as index
UST_combined['time'] = pd.to_datetime(UST_combined['time'], utc=True)
UST_combined.set_index('time', inplace=True)
UST_combined.index = UST_combined.index.tz_convert('Europe/Zurich')

Fetching 2Y UST data...
Fetching 10Y UST data...


In [140]:
plot_yield_curve_regimes_plotly(classify_regime(UST_combined), "UST")    

In [141]:
#Pull intraday data
print("Fetching 2Y Bund data...")
Bund_2y = get_intraday_data("GDBR2 Index", start_datetime, end_datetime)
print("Fetching 10Y bund data...")
Bund_10y = get_intraday_data("GDBR10 Index", start_datetime, end_datetime)

#Merge dataframes on time
Bund_combined = pd.merge(Bund_2y[['time', 'close']], Bund_10y[['time', 'close']], 
                      on='time', suffixes=('_2y', '_10y'))

#Set time as index
Bund_combined['time'] = pd.to_datetime(Bund_combined['time'], utc=True)
Bund_combined.set_index('time', inplace=True)
Bund_combined.index = Bund_combined.index.tz_convert('Europe/Zurich')

Fetching 2Y Bund data...
Fetching 10Y bund data...


In [142]:
plot_yield_curve_regimes_plotly(classify_regime(Bund_combined), "Bund")    